In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
from scipy import stats
import math
from sklearn import tree,linear_model,neighbors
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.ensemble import RandomForestRegressor
%matplotlib inline
import warnings 
warnings.filterwarnings("ignore")
sns.set(style="white", color_codes=True)
sns.set_context(rc={"font.family":'sans',"font.size":10,"axes.titlesize":24,"axes.labelsize":24}) 
from sklearn import model_selection

# Upload data (customers model)

In [4]:
train=pd.read_csv('TrainClean.csv',index_col=0).reset_index(drop=True)
#test=pd.read_csv('TestClean.csv',index_col=0).reset_index(drop=True)

columnscustomers_lasso=['HasPromotions', 'NearestCompetitor', 'Region_AreaKM2',
       'Mean_Dew_PointC', 'Mean_Sea_Level_PressurehPa', 'Precipitationmm',
       'StandardMarket', 'HyperMarket', 'ShoppingCenter', 'General',
       'WithFishDepartment', 'Region0', 'Region1', 'Region3', 'Region4',
       'Region5', 'Region6', 'Region7', 'Region8', 'Region9', 'January',
       'February', 'April', 'May', 'June', 'September', 'October', 'November',
       'December', 'Wednesday', 'Friday', 'Saturday', 'Monday', 'Thursday',
       'Fog', 'Snow']
columnssales_lasso=['IsHoliday', 'HasPromotions', 'NearestCompetitor', 'Region_AreaKM2',
       'Region_PopulationK', 'Mean_Humidity', 'Mean_Sea_Level_PressurehPa',
       'Mean_TemperatureC', 'Mean_VisibilityKm', 'Precipitationmm',
       'StandardMarket', 'HyperMarket', 'SuperMarket', 'ShoppingCenter',
       'WithFishDepartment', 'WithNonFoodDepartment', 'Region0', 'Region1',
       'Region2', 'Region3', 'Region4', 'Region6', 'Region7', 'Region8',
       'January', 'February', 'March', 'April', 'May', 'June', 'July',
       'September', 'October', 'November', 'December', 'Tuesday', 'Friday',
       'Saturday', 'Monday', 'Thursday', 'Sunday', 'Fog', 'Snow', 'Nothing',
       'Rain', 'Thunderstorm', 'NumberOfCustomers']
columnsdirect_lasso=['HasPromotions', 'NearestCompetitor', 'Region_GDP', 'Mean_Dew_PointC',
       'Mean_Sea_Level_PressurehPa', 'Precipitationmm', 'HyperMarket',
       'ShoppingCenter', 'General', 'Region0', 'Region2', 'Region3', 'Region4',
       'Region5', 'Region6', 'Region8', 'Region9', 'January', 'May', 'June',
       'September', 'October', 'November', 'December', 'Tuesday', 'Wednesday',
       'Friday', 'Saturday', 'Monday']
columnscustomers_rfs=['NearestCompetitor', 'ShoppingCenter', 'Region3', 'HasPromotions',
       'StandardMarket', 'Region7', 'General', 'Region_AreaKM2', 'Region_GDP',
       'WithNonFoodDepartment', 'HyperMarket', 'SuperMarket', 'Monday',
       'December', 'Saturday', 'Region8', 'Region_PopulationK', 'Region0',
       'Region9', 'Region2', 'Mean_Sea_Level_PressurehPa', 'Mean_Humidity',
       'Region4', 'Mean_Dew_PointC', 'Mean_VisibilityKm']
columnssales_rfs=['NumberOfCustomers', 'HasPromotions', 'NearestCompetitor',
       'StandardMarket', 'Monday', 'General', 'December', 'Region_AreaKM2',
       'Region3', 'Region_GDP', 'Region_PopulationK', 'WithNonFoodDepartment',
       'HyperMarket', 'Region2', 'ShoppingCenter', 'Saturday', 'Region9',
       'SuperMarket', 'Region6', 'Region5', 'Region7', 'Region0',
       'WithFishDepartment', 'Mean_Sea_Level_PressurehPa', 'Mean_VisibilityKm']
direcly_sales_rfs = ['NearestCompetitor', 'HasPromotions', 'General', 'HyperMarket',
       'December', 'Region3', 'Monday', 'Region_GDP', 'ShoppingCenter',
       'Region_AreaKM2', 'Saturday', 'Region_PopulationK', 'StandardMarket',
       'SuperMarket', 'Mean_Sea_Level_PressurehPa', 'Region9',
       'Mean_VisibilityKm', 'Mean_Humidity', 'Region7', 'Mean_Dew_PointC', 'H',
       'Mean_Wind_SpeedKm_h', 'Mean_TemperatureC', 'WCI', 'Region5']

# Function definition

In [14]:
def AverageModelsCrossvalidation(dataset, features, target, no_boosting_runs):
    
    dataset_size = dataset.shape[0]
    crossperf = []
    r2mean = []
    dataset = dataset.sample(frac=1,random_state=1234)
    k = 3
    step = math.floor(dataset_size/k)
    
    for i in range(0,dataset_size,step+1):
        perf = []
        model_list = []
        
        train_data, test_data = DataSplit(dataset,i,step)
    
        x = train_data[features]
        y_residual = train_data[target].values.reshape(-1,1)    
        x_test = test_data[features]
        y_test = test_data[target].values.reshape(-1,1)
    
        for i in range(no_boosting_runs):
            y_residual,yp_boosting = DT_FittingAndEvaluation(x, x_test, y_residual, y_test, perf, model_list)
        
        y =  train_data[target]
        model1 = RandomForestRegressor(random_state=12345) 
        model1.set_params(n_estimators=150,min_samples_leaf=5)
        model1.fit(x,y)
        
        yp_forest = model1.predict(x_test)
        
        mean_predictions = []
        
        for i in range(yp_forest.shape[0]):
            mean_predictions.append((yp_boosting[i]+yp_forest[i])/2)
    
        r2mean.append(r2_score(y_test,mean_predictions))
        crossperf.append(perf)
    
    return crossperf,r2mean

In [5]:
def DataSplit(dataset,i,step):
    left_data = dataset[0:i]
    test_data = dataset[i:i+step]
    right_data = dataset[i+step+1:dataset.shape[0]]        
    train_data = left_data.append(right_data)
    train_data = train_data.reset_index(drop=True)
    return train_data, test_data

In [6]:
def DT_FittingAndEvaluation(x, x_test, y_residual, y_test, perf, model_list):
    y_residual = DT_Fitting(x, y_residual, model_list)
    result=GradientBoosting_eval(model_list,x_test)
    perf.append(r2_score(y_test,result))
    perf[-20:]
    return y_residual,result

In [7]:
def DT_Fitting(x, y_residual, model_list):
    clf = tree.DecisionTreeRegressor(max_depth=6)
    clf = clf.fit(x, y_residual)
    model_list.append(clf)
    yp = clf.predict(x).reshape(-1,1)
    y_residual = y_residual-yp
    return y_residual

In [8]:
def GradientBoosting_eval(models,data_to_predict):
    prediction = np.zeros((len(data_to_predict),1))
    for model in models:
        yp = model.predict(data_to_predict).reshape(-1,1)
        prediction += yp
    return prediction.reshape(-1,1)

In [9]:
def GradientBoosting_performances(crossperf):
    finalperf = []
    for i in range(len(crossperf[0])):
        perflist=[]
        for performance in crossperf:
            perflist.append(performance[i])
        finalperf.append(np.average(perflist))
        del perflist
        print("Boosting %d : %d modelli - R2: %f\n" % (i,i+1,finalperf[i]))
    plt.plot(finalperf)

# Predict Sales direclty

### No Features Selection

In [12]:
all_features = train.drop(columns=['NumberOfSales','NumberOfCustomers']).columns

In [15]:
crossperf,meanperf = AverageModelsCrossvalidation(train,all_features,'NumberOfSales',130)

In [25]:
np.average([crossperf[0][129] , crossperf[1][129] , crossperf[2][129]])

0.8483283792414286

In [26]:
np.average(meanperf)

0.8681417308283904